In [63]:
import pathlib
import pandas as pd
import numpy as np
from os import path

In [64]:
curr_dir_path = str(pathlib.Path().absolute())
data_path = curr_dir_path + "/Data/"

In [11]:
import numpy as np
import pathlib
import pandas as pd
from os import path

curr_dir_path = str(pathlib.Path().absolute())
data_path = curr_dir_path + "/Data/"

def start_end_positions(file="chr21_annotations.csv"):
    '''
    Function to read annotation given file for chromosome and get [start, end] base pair position of genes
    :param file:
            chromosome annotation file
    :return: ndarray (num_genes, 2)
            Array containing [start, end] base pair position on chromosome for each gene.
    '''
    final_chr = pd.read_csv(data_path + file, sep='\t')
    chr_gene = final_chr[final_chr['type'] == 'gene']
    start_end_pos = pd.DataFrame(
        {'start_pos': chr_gene['start'], 'end_pos': chr_gene['end']})  # shape: (875,2) for chr21
    start_end_pos.reset_index(drop=True, inplace=True)
    return start_end_pos


def preprocess_chromosomeseq_file(chr_file="chr21.fa", write_to_file=False):
    '''
    Function to remove special characters in chromosome sequence
    :param chr_file: File containing nucleotide sequence for chromosome
    :param write_to_file: bool, optional
            Variable to control writing extracted information to txt file
    :return: str
            String containing nucleotide sequence for chromosome
    '''
    file_object = open(data_path + chr_file, "r")

    chrm = file_object.read()
    # print('Length of unprocessed chromosome: '+len(chrm))  # 47644190

    chrm = chrm.replace('\n', '')
    #Fasta file contains name of chromosome (eg. '>chr21') at the start: removing it
    if (chrm[5].isdigit()):
        chrm = chrm[6:]
    else:
        chrm = chrm[5:]
    # print('Length of processed chromosome: ', len(chrm))  # 46709983

    #Append a blank at the beginning and end of string (since annotations are 1-indexed
    chrm = ' '+chrm+' '   # len = 46709985
    print('Length of processed chromosome: ', len(chrm))
    if write_to_file or not path.exists(data_path+chr_file.replace("fa", "txt")):
        text_file = open(data_path + chr_file.replace("fa", "txt"), "w")
        text_file.write(chrm)
        text_file.close()
    return chrm


def create_chromosome_annotations(chrm='chr21', write_to_file=False):
    '''
    Function to read annotation file for human genome sequence GRCh38 and get annotations for specified chromosome
    :param chrm: str
            chromosome information to be extracted
    :param write_to_file: bool, optional
            Variable to control writing extracted information to file
    :return: None
    '''
    ann_file = "gencode.v34.annotation.gtf"
    col_names = ['chr_name', 'source', 'type', 'start', 'end', '.', 'strand', ',', 'other']
    ann = pd.read_csv(data_path + ann_file, sep='\t', header=None)
    ann.columns = col_names

    # print(ann.shape)  # (2912496, 9)
    # print(ann.head())

    chr_no = ann[ann['chr_name'] == chrm]
    del chr_no['.']
    del chr_no[',']
    #print(chr_no.head())

    other = list(map(lambda x: x.split(';'), chr_no['other']))
    df_other = pd.DataFrame(other)
    df_other[0] = list(map(lambda x: x[8:].strip('"'), df_other[0]))
    df_other[1] = list(map(lambda x: x[14:].strip('"') , df_other[1]))
    del chr_no['other']

    chr_no.reset_index(drop=True, inplace=True)
    df_other.reset_index(drop=True, inplace=True)
    
    final_chr = pd.concat([chr_no, df_other], axis=1)
    print(final_chr.head())
    if write_to_file or not path.exists(data_path + chrm + '_annotations.csv'):
        final_chr.to_csv(data_path + chrm + '_annotations.csv', index=False, sep='\t')


if __name__ == "__main__":
    #chrm = preprocess_chromosomeseq_file("chr21.fa", write_to_file=False)  #read fasta file for chromosome, which generate the txt files
    create_chromosome_annotations(chrm='chr21', write_to_file=True) #get annotations for particular chromosome
    #file_object = open(data_path + "chr21.txt", "r")
    #chrm = file_object.read()
    #start_end_pos = start_end_positions("chr21_annotations.csv")
    #print(start_end_pos.shape)

  chr_name  source        type    start      end strand                  0  \
0    chr21  HAVANA        gene  5011799  5017145      +  ENSG00000279493.1   
1    chr21  HAVANA  transcript  5011799  5017145      +  ENSG00000279493.1   
2    chr21  HAVANA        exon  5011799  5011874      +  ENSG00000279493.1   
3    chr21  HAVANA         CDS  5011799  5011874      +  ENSG00000279493.1   
4    chr21  HAVANA        exon  5012548  5012687      +  ENSG00000279493.1   

                     1                            2                        3  \
0         otein_coding       gene_name "FP565260.4"                  level 2   
1   "ENST00000624081.1   gene_type "protein_coding"   gene_name "FP565260.4"   
2   "ENST00000624081.1   gene_type "protein_coding"   gene_name "FP565260.4"   
3   "ENST00000624081.1   gene_type "protein_coding"   gene_name "FP565260.4"   
4   "ENST00000624081.1   gene_type "protein_coding"   gene_name "FP565260.4"   

   ...                         12                 

In [11]:
def get_genes_exons_df(chrm, start_end_pos, write_to_file=False):
    '''
    Function to read positions and chromosome file and return the gene sequences
    :param chrm: str
            Nucleotide sequence of chromosome
    :param start_end_pos: data frame
            Start and end positions of each gene in the chromosome
    :param write_to_file: bool, optional
            Variable to control writing extracted information to file
    :return: data frame (num_genes, 3)

    '''
    indices = list(range(0, len(start_end_pos)))
    column_names = ['gene_sequence', 'gene_ranges', 'exon_ranges']
    df = pd.DataFrame(index=indices, columns=column_names)

    # get -300 and +300 nucleotides around each gene
    df['gene_sequence'] = list(map(lambda x: chrm[start_end_pos['start_pos'][x]-300:start_end_pos['end_pos'][x]+300]
                                   , indices) )
    df['gene_ranges'] = list(zip(start_end_pos['start_pos'], start_end_pos['end_pos']))

    gene_lists = list(map(lambda x: df.iloc[x]['gene_sequence'], indices))
    df['exon_ranges'] = list(map(lambda x: get_exon_boundaries(x), gene_lists))

    exon_lengths = [[x[1] - x[0] for x in ranges] for ranges in df['exon_ranges'] if (len(ranges) > 0)]
    max_exon_lengths = list(map(max, exon_lengths))
    min_exon_lengths = list(map(min, exon_lengths))
    no_of_exons = list(map(len, exon_lengths))



    meta_info = pd.DataFrame(list(zip(no_of_exons, max_exon_lengths, min_exon_lengths, exon_lengths)),
                      columns=['no_of_exons', 'max_exon_length', 'min_exon_length', 'exon_lengths'])
    meta_info.to_csv(data_path + 'chr21_exons_info.csv', index=False, sep='\t')
    if write_to_file:  # if file doesn't exist
        df.to_csv(data_path + 'chr21_genes_exons.csv', index=False, sep='\t')
    return df

In [12]:
def start_end_positions(file="chr21_annotations.csv"):
    """
    Function to read annotation given file for chromosome and get [start, end] base pair position of genes
    :param file:
            chromosome annotation file
    :return: ndarray (num_genes, 2)
            Array containing [start, end] base pair position on chromosome for each gene.
    """
    final_chr = pd.read_csv(data_path + file, sep='\t')
    chr_gene = final_chr[final_chr['type'] == 'gene']
    start_end_pos = pd.DataFrame(
        {'start_pos': chr_gene['start'], 'end_pos': chr_gene['end']})  # shape: (875,2) for chr21
    start_end_pos.reset_index(drop=True, inplace=True)
    return start_end_pos

In [13]:
def preprocess_chromosomeseq_file(chr_file="chr21.fa", write_to_file=False):
    '''
    Function to remove special characters in chromosome sequence
    :param chr_file: File containing nucleotide sequence for chromosome
    :param write_to_file: bool, optional
            Variable to control writing extracted information to file
    :return: str
            String containing nucleotide sequence for chromosome
    '''
    file_object = open(data_path + chr_file, "r")

    chrm = file_object.read()
    chrm = chrm.replace('\n', '')
    # print(len(chr21))  # 47644190
    if write_to_file:
        text_file = open(data_path + chr_file.replace("fa", "txt"), "w")
        text_file.write(chrm)
        text_file.close()
    return chrm

In [13]:
import pathlib
import pandas as pd
from os import path
import json
import itertools

curr_dir_path = str(pathlib.Path().absolute())
data_path = curr_dir_path + "/Data/"

def gene_start_end_positions(chrm_ann):
    '''
    Function to read annotation for given chromosome and get [start, end] base pair position of genes
    :param chrm_ann: data frame
            Annotations for given chromosome
    :return: ndarray (num_genes, 2)
            Array containing [start, end] base pair position on chromosome for each gene.
    '''
    chr_gene = chrm_ann[chrm_ann['type'] == 'gene']
    gene_start_end_pos = pd.DataFrame(
        {'start_pos': chr_gene['start'], 'end_pos': chr_gene['end']})  # shape: (875,2) for chr21
    gene_start_end_pos.reset_index(drop=True, inplace=True)
    print('Start,end shape:', gene_start_end_pos.shape)
    return gene_start_end_pos

def get_indices_of_table(df, last_index):
    '''
    Function to
    :param df: data frame
    :return: list of indexes of the data frame
    '''
    indices = pd.Index.to_list(df.index)
    indices.append(last_index)

    return indices

def no_transcripts_per_gene(cur, nex, chrm_ann):
    '''
    Function to
    :param cur:
    :param nex:
    :param chrm_ann:
    :return:
    '''
    gene_table = chrm_ann.iloc[cur:nex]
    transcript_counts_per_gene=gene_table['type'].value_counts()['transcript']

    return transcript_counts_per_gene

def get_chunk(cur, nex, chrm_ann, type):
    '''
    Function to get subset of the annotation data frame according to condition
    :param cur: int
            Start index
    :param nex: int
            End index
    :param chrm_ann: data frame
            Annotation data frame
    :return: data frame
            Subset of data frame
    '''
    table = chrm_ann.iloc[cur:nex]
    df = table[table['type'] == type]
    return df

def create_dict(chrm_seq, chrm_ann):
    '''
    Function to create the json file
    :param chrm_seq: string
            Nucleotide sequence of the chromosome
    :param chrm_ann: data frame
            Annotation file for the chromosome
    :return: dictionary
        Dictionary to be stored as the json file
    '''
    chrm_gene = chrm_ann[chrm_ann['type'] == 'gene']

    gene_ids = list(chrm_gene['0'])  #[l.strip('"') for l in

    gene_start_end_pos = gene_start_end_positions(chrm_ann)
    gene_bounds = list(zip(gene_start_end_pos.start_pos, gene_start_end_pos.end_pos))

    gene_strand = list(chrm_gene['strand'])
    indices = list(range(0, len(gene_start_end_pos)))
    gene_sequence = list(map(lambda x: chrm_seq[gene_start_end_pos['start_pos'][x]:gene_start_end_pos['end_pos'][x]]
            , indices))
    gene_indices = get_indices_of_table(chrm_gene, len(chrm_ann))

    my_dictionary = {'main': []}

    for i in range(0,len(chrm_gene)):
        cur = gene_indices[i]
        nex = gene_indices[i+1]
        lis = {}

        lis.update({'gene_id': gene_ids[i]})
        lis.update({'gene_strand': gene_strand[i]})
        lis.update({'gene_bounds': gene_bounds[i]})
        lis.update({'gene_sequence':gene_sequence[i]})  #take into account reverse complementarity

        no_transcripts = no_transcripts_per_gene(cur, nex, chrm_ann)
        lis.update({'no_of_transcripts': int(no_transcripts)})

        transcripts = []
        transcript = get_chunk(cur, nex, chrm_ann, 'transcript')
        transcript_ids = [l.strip('"') for l in list(transcript['1'])]
        transcript_ranges = list(zip(transcript['start'], transcript['end']))
        transcript_indices = get_indices_of_table(transcript, nex)

        for j in range(0,len(transcript_ranges)):

            list_transcript = {}

            list_transcript.update({'transcript_id': transcript_ids[j]})
            list_transcript.update({'transcript_range': transcript_ranges[j]})

            exon = get_chunk(transcript_indices[j], transcript_indices[j+1], chrm_ann, 'exon')
            exon_ranges = list(zip(exon['start'], exon['end']))

            list_transcript.update({'no_of_exons': int(len(exon_ranges))})
            exons = []
            list_exon_range = {}

            for er in exon_ranges:
                list_exon_range.update({'exon_ranges': er})
                exons.append({'exon_ranges': er})

            list_transcript.update({'exons': exons})
            transcripts.append(list_transcript)

        lis.update({'transcripts': transcripts})
        my_dictionary['main'].append(lis)

    return my_dictionary

if __name__ == "__main__":

    chrm_txt_file = "chr21.txt"
    chrm_ann_file = "chr21_annotations.csv"

    file_object = open(data_path + chrm_txt_file, "r")
    chrm_seq = file_object.read()
    chrm_ann = pd.read_csv(data_path + chrm_ann_file, sep='\t')

    #with open(data_path+"data.json") as f:
    #        data = json.load(f)
    my_dictionary = create_dict(chrm_seq, chrm_ann)
    with open(data_path+chrm_txt_file.replace('.txt','')+'_data.json', 'w') as file:
        json.dump(my_dictionary, file)

/Users/asmitapoddar/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Start,end shape: (875, 2)


In [68]:
def gene_start_end_positions(chrm_ann):
    '''
    Function to read annotation for given chromosome and get [start, end] base pair position of genes
    :param chrm_ann: data frame
            Annotations for given chromosome
    :return: ndarray (num_genes, 2)
            Array containing [start, end] base pair position on chromosome for each gene.
    '''
    chr_gene = chrm_ann[chrm_ann['type'] == 'gene']
    gene_start_end_pos = pd.DataFrame(
        {'start_pos': chr_gene['start'], 'end_pos': chr_gene['end']})  # shape: (875,2) for chr21
    gene_start_end_pos.reset_index(drop=True, inplace=True)
    print('Start,end shape:', gene_start_end_pos.shape)
    return gene_start_end_pos

In [69]:
chrm_txt_file = "chr21.txt"
chrm_ann_file = "chr21_annotations.csv"

file_object = open(data_path + chrm_txt_file, "r")
chrm_seq = file_object.read()
chrm_ann = pd.read_csv(data_path + chrm_ann_file, sep='\t')

/Users/asmitapoddar/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [70]:
df = gene_start_end_positions(chrm_ann)

Start,end shape: (875, 2)


In [71]:
df.head()

,start_pos,end_pos
0,5011799,5017145
1,5022493,5040666
2,5073458,5087867
3,5079294,5128413
4,5116343,5133805


In [96]:
chrm_gene = chrm_ann[chrm_ann['type'] == 'gene']

list(chrm_gene.columns)

['chr_name',
 'source',
 'type',
 'start',
 'end',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21']

In [103]:
 gene_ids = [l.strip('"') for l in list(chrm_gene['0'])]

In [98]:
gene_start_end_pos = gene_start_end_positions(chrm_ann) 
l = list(zip(gene_start_end_pos.start_pos, gene_start_end_pos.end_pos))

Start,end shape: (875, 2)


In [100]:
indices = list(range(0, len(gene_start_end_pos)))
gene_sequence = list(map(lambda x: chrm_seq[gene_start_end_pos['start_pos'][x]:gene_start_end_pos['end_pos'][x]]
            , indices))